In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch
import re
import string
import codecs
import unidecode


import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords
import nltk
from string import punctuation
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC


In [2]:
train_data = pd.read_csv('data/train.csv',encoding='utf-8')
test_data = pd.read_csv('data/test.csv',encoding='utf-8')

In [3]:
train_data.dropna(inplace=True)
train_data.isna().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [4]:
from bs4 import BeautifulSoup
def remove_html_encodings(x):
    example1 = BeautifulSoup(x, 'lxml')
    return example1.get_text()

In [5]:
#train_data["tweet"] = train_data["tweet"].apply(lambda x: remove_html_encodings(x))

In [6]:
#test_data['tweet'] = test_data['tweet'].apply(lambda x: remove_html_encodings(x))

In [7]:
train_data = train_data[train_data['sentiment'] != 3]

In [8]:
# Removing Stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

stopwords = set(STOPWORDS)
more_stopwords = {'amp', "quot","via","will"}
stopwords = stopwords.union(more_stopwords)

included_stop_words = ['nobody','not','n‘t','less',"n't",'no','never','cannot','full','none','first','least','top','nothing','against']

new_stop_words = set(i for i in stopwords if i not in included_stop_words)
new_stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'all',
 'also',
 'am',
 'amp',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 "can't",
 'com',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'else',
 'ever',
 'few',
 'for',
 'from',
 'further',
 'get',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'hence',
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'however',
 'http',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'k',
 "let's",
 'like',
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'nor',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'otherwise',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',


In [9]:
#train_data["tweet"] = train_data["tweet"].apply(lambda x : " ".join([t for t in x.split() if t not in new_stop_words]))
#test_data["tweet"] = test_data["tweet"].apply(lambda x : " ".join([t for t in x.split() if t not in new_stop_words]))

In [10]:
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
train_data['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in train_data['tweet']] 
print(train_data['tokenized_text'].head(10))

0    [sxswnui, sxsw, apple, defining, language, of,...
1    [learning, ab, google, doodles, all, doodles, ...
2    [one, of, the, most, in, your, face, ex, of, s...
3    [this, iphone, sxsw, app, would, pretty, aweso...
4    [line, outside, the, apple, store, in, austin,...
5    [technews, one, lone, dude, awaits, ipad, at, ...
6    [sxsw, tips, prince, npr, videos, toy, shoppin...
7    [nu, user, rt, mention, new, ubersocial, for, ...
8    [free, sxsw, sampler, on, itunes, link, freemu...
9    [think, might, go, all, weekend, without, seei...
Name: tokenized_text, dtype: object


In [11]:
test_data['tokenized_text'] = [simple_preprocess(line,deacc=True) for line in test_data['tweet']]
print(test_data['tokenized_text'].head(10))

0    [audience, what, prototyping, tools, do, you, ...
1    [at, sxsw, send, your, best, photos, amp, vide...
2    [mention, and, here, pic, of, you, winning, yo...
3    [google, marissa, mayer, mobile, phone, as, cu...
4    [sxsw, google, maps, is, even, cooler, than, t...
5    [rt, mention, in, front, of, mention, popup, s...
6    [rt, mention, in, my, next, life, coming, back...
7    [google, celebrating, pi, day, in, style, at, ...
8    [hmmm, is, it, bit, weird, that, sxsw, is, not...
9    [mention, to, launch, circles, later, today, a...
Name: tokenized_text, dtype: object


In [12]:
# porter = PorterStemmer()
lemmatizer =  WordNetLemmatizer()
# stemming words
# stemmer = PorterStemmer()
corpus = []
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x:[lemmatizer.lemmatize(i) for i in x])
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x:' '.join(x))
for i in train_data.tokenized_text:
    corpus.append(i)
print(corpus)

['sxswnui sxsw apple defining language of touch with different dialect becoming smaller', 'learning ab google doodle all doodle should be light funny amp innovative with exception for significant occasion googledoodle sxsw', 'one of the most in your face ex of stealing the show in yr rt mention quot at sxsw apple school the mkt expert quot link', 'this iphone sxsw app would pretty awesome if it didn crash every min during extended browsing fuckit illmakeitwork', 'line outside the apple store in austin waiting for the new ipad sxsw link', 'technews one lone dude awaits ipad at apple uªs sxsw store link tech_news apple ipad_ sxsw tablet tech', 'sxsw tip prince npr video toy shopping with zuckerberg link sxsw ipad', 'nu user rt mention new ubersocial for iphone now in the app store includes uberguide to sxsw sponsored by mashable', 'free sxsw sampler on itunes link freemusic', 'think might go all weekend without seeing the same ipad case twice sxsw', 'it official buying an ipad sxsw eleva

In [13]:
test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x:[lemmatizer.lemmatize(i) for i in x])
test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x:' '.join(x))

In [14]:
from textblob import TextBlob
# Creating Polarity Column using TextBlob
tb_polarity = []
for sentence in train_data['tokenized_text']:
    temp = TextBlob(sentence)
    tb_polarity.append(temp.sentiment[0])
#train_data['polarity'] = tb_polarity

test_polarity =[]
for sentence in test_data['tokenized_text']:
    temp = TextBlob(sentence)
    test_polarity.append(temp.sentiment[0])
# print(tb_polarity)
#test_data['polarity'] = test_polarity

In [15]:
train_data['polarity'].isnull().sum()

KeyError: 'polarity'

In [16]:
test_data['polarity'].isnull().sum()

KeyError: 'polarity'

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tweets_train = tfidf_vectorizer.fit_transform(train_data['tokenized_text'])

In [23]:
tweets_test = tfidf_vectorizer.transform(test_data['tokenized_text'])

In [28]:
X=tweets_train.toarray()
X = pd.DataFrame(X)
print(X.shape, train_data.shape)

X.index = train_data.index

#X['polarity'] = train_data['polarity']
#train_data['sentiment']

(7148, 7668) (7148, 4)


In [30]:
test=tweets_test.toarray()
test = pd.DataFrame(test)
print(test.shape, test_data.shape)

test.index = test_data.index

#test['polarity'] = test_data['polarity']
#train_data['sentiment']

(1819, 7668) (1819, 3)


In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,train_data.sentiment, test_size=0.25,random_state=22)

from imblearn.over_sampling import SMOTE

# Instantiate smote
smote = SMOTE(random_state=9)

# fit_sample onm training data
X_smote,y_smote = smote.fit_sample(X_train,y_train)

svc_model_smote = SVC(C=1, gamma = 1, kernel = 'rbf')
svc_model_smote.fit(X_smote,y_smote)
pred = svc_model_smote.predict(X_test)
score_svc  = f1_score(y_test, pred, average='weighted')
print(score_svc)

0.6768598965787869


In [32]:
ID = test_data['tweet_id']
len(ID)

1819

In [33]:
pred = svc_model_smote.predict(test)

In [34]:
print(pred, len(pred))
prediction = pd.DataFrame(pred,columns=['sentiment'])
prediction

[1 1 1 ... 1 1 1] 1819


sentiment
0             1
1             1
2             1
3             2
4             2
...         ...
1814          1
1815          1
1816          1
1817          1
1818          1

[1819 rows x 1 columns]

In [35]:
sub_2 = pd.concat([ID,prediction['sentiment']],1)
sub_2

tweet_id  sentiment
0         7506          1
1         7992          1
2          247          1
3         7688          2
4         3294          2
...        ...        ...
1814      1550          1
1815      1933          1
1816      9052          1
1817      4219          1
1818      7210          1

[1819 rows x 2 columns]

In [ ]:
# sub_2.to_csv('sub_2.csv',index=False)